In [ ]:
!pip install tf-explain==0.1.0

Upload cat.jpg and dog.jpg images

In [ ]:
# You can upload multiple files at a time
from google.colab import files
files.upload()

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import get_file
import json

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tf_explain.core.grad_cam import GradCAM

import PIL
from PIL import Image, ImageDraw, ImageFont

import matplotlib.cm as cm

from argparse import ArgumentParser

import glob
import os

CLASS_INDEX = None
CLASS_INDEX_PATH = 'imagenet_class_index.json'

In [ ]:
!curl https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json --output imagenet_class_index.json

In [ ]:
# Note: decode_predictions(preds, top) is originally a keras function.
# We have modified it here so that it returns the index of the class label along with the predictions.
# The results are assimilated based on the assumption that there is only one top 1% prediction.

def decode_predictions_modified(preds, top=1):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError(
            '`decode_predictions` expects ' 'a batch of predictions ''(i.e. a 2D array of shape (samples, 1000)). ' 'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        for i in top_indices:
            results = [i, tuple(CLASS_INDEX[str(i)]), (pred[i],)]
    return results

# Function that takes an image and model and produces the predictions

def get_predictions(img, model):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return decode_predictions_modified(preds, top=1)

# NOTE: These two functions are taken from Shao-Chuan Wang <shaochuan.wang AT gmail.com> 
# as per the copyright on http://code.activestate.com/recipes/577591-conversion-of-pil-image-and-numpy-array/

# Modified from the original format to take only the array as input and calculate the size on the fly
def array_to_PIL(arr):
    mode = 'RGBA'
    # Use only the height and width for further processing
    size = (arr.shape[0], arr.shape[1])  # (224,224)
    arr = arr.reshape(arr.shape[0]*arr.shape[1], arr.shape[2])
    if len(arr[0]) == 3:
        arr = np.c_[arr, 255*np.ones((len(arr), 1), np.uint8)]
    return Image.frombuffer(mode, size, arr.tostring(), 'raw', mode, 0, 1)

# Function that puts text based prediction and class name on top of the image
def overlay_prediction_on_image(img, prediction_class, prediction_probability, width, height):
    img = img.resize((width, height), Image.ANTIALIAS)
    draw = ImageDraw.Draw(img)
    l = len(prediction_class)
    # Place a black rectangle to provide a background for the text
    # The size of the rectangle should change with respect to the image
    draw.rectangle([int(width*0.05), int(width*0.05),
                    int(width*0.5), int(width*0.11)], fill=(0, 0, 0))
    draw.text((int(width*0.06), int(width*0.06)), '{0:.0f}'.format(
        prediction_probability) + "% " + prediction_class, fill=(255, 255, 255))
    return img

# Based on StackOverflow code by user DTing
# https://stackoverflow.com/questions/30227466/combine-several-images-horizontally-with-python

def join_images(img1, img2):
    widths, heights = zip(*(i.size for i in [img1, img2]))
    total_width = sum(widths)
    max_height = max(heights)
    new_img = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for img in [img1, img2]:
        new_img.paste(img, (x_offset, 0))
        x_offset += img.size[0]
    return new_img

def process_image(image_path, output_path):
    model = VGG16(weights='imagenet', include_top=True, input_tensor=None, input_shape=None, pooling=None, classes=1000)
    explainer = GradCAM()

    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    data = ([img], None)

    original_image = Image.open(image_path)
    width = int(original_image.size[0]/4)
    height = int(original_image.size[1]/4)
    original_image.thumbnail((width, height), Image.ANTIALIAS)

    class_index, class_name, prob_value = get_predictions(img, model)
    heatmap = explainer.explain(data, model, "block5_conv3", class_index)

    # overlay the text prediction on the heatmap overlay
    heatmap_with_prediction_overlayed = overlay_prediction_on_image(
        array_to_PIL(heatmap), class_name[-1], prob_value[0] * 100, width, height)

    # place the images side by side
    joined_image = join_images(
        original_image, heatmap_with_prediction_overlayed)
    joined_image.save(output_path)

# **What Does My Neural Network Think?**

Visualizing the Heatmap of an Image

In [ ]:
process_image("dog.jpg", "dog_output.jpg")
process_image("cat.jpg", "cat_output.jpg")

In [ ]:
Image.open('dog_output.jpg')

In [ ]:
Image.open('cat_output.jpg')